If any problems occur, please refer to the E2_Vector_Search to check out the set up process. 

In [1]:
# Import Vertex AI SDK
PROJECT_ID = !gcloud config get project
PROJECT_ID = PROJECT_ID.n
LOCATION = "europe-west2"
LOCATION_DEPLOY = "europe-west2" #Location to deploy GCP resources

import vertexai
from google.cloud import aiplatform

vertexai.init(project=PROJECT_ID, location=LOCATION)

2023-12-05 17:30:07.563339: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-05 17:30:08.734106: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-12-05 17:30:08.734232: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/loca

In [2]:
from vertexai.preview.language_models import ChatModel

""" 
between using the most stable model in the last 6 month: chat-bison@001 
and the latest, most capable model, there is a massive difference in 
terms of context awareness, length of answer, ability to follow instructions,
etc. How do we measure these using metrics? I'm working on it.

I lied, the latest model, whilst sometimes its answers can be amazing,
definitely not as good when using RAG, it ignores context too often,
until I fix that, we'll work with the stable version.

"""
class PaLMWrapper:
    def __init__(self):
        self.chat_model = ChatModel.from_pretrained("chat-bison@001")
        self.parameters = {
            "temperature": 0.3,
            "max_output_tokens": 256,
            "top_p": 0.95,
        }
    
    def generate_response(self, context, message):
        chat = self.chat_model.start_chat(context=context)
        response = chat.send_message(message, **self.parameters)
        return response.text

In [3]:
pw = PaLMWrapper()

In [4]:
import pandas as pd

import vertexai
from vertexai.preview.language_models import TextEmbeddingModel
from google.cloud import aiplatform

class VertexAIVectorStore:
    def __init__(self):
        
        # god awful way of doing it, should be a config and passed through but oh well hacky hack
        self.gen_ai_index_endpoint = aiplatform.MatchingEngineIndexEndpoint(
            index_endpoint_name="projects/playpen-8d8611/locations/europe-west2/indexEndpoints/5891412000042385408"
        )

        self.gen_ai_index = aiplatform.MatchingEngineIndex(
            index_name="projects/playpen-8d8611/locations/europe-west2/indexes/2680908415680643072"
        )
        
        self.model = TextEmbeddingModel.from_pretrained("textembedding-gecko@001")
        self.df = pd.read_csv('text_data_g_embedding.csv')
        
    def search(self, input, k=3):
        embedding_vec =  self.model.get_embeddings([input])[0].values #Send request to embedding model to generate the embedding vector

        #find neighbours using vector search
        neighbours = self.gen_ai_index_endpoint.find_neighbors(
            deployed_index_id="gen_ai_deployed_index",
            queries=[embedding_vec],
            num_neighbors=k,
        )[0]
        
        results = []
        for nb in neighbours:
            nb_id = int(nb.id)
            if nb_id < len(self.df):
                url = self.df.iloc[int(nb.id)]['url']
                text = self.df.iloc[int(nb.id)]['text']
                score = nb.distance
                results.append((url, text, score))
            else:
                results=[('', '', 0)]
        
        return results


In [5]:
vai = VertexAIVectorStore()

In [9]:
# couldnt get langchain to work so custom RAG anyone?
import time

class RAG:
    def __init__(self, vector_store, palm_wrapper, initial_system_prompt=True):
        self.vector_store = vector_store
        self.palm_wrapper = palm_wrapper
        # the comment from mathew regarding memory
        self.conversation_history = []
        
        if initial_system_prompt:            
            system_prompt = r"You are a professional assistant with extensive experience helping numerous small and medium businesses. You work for a large retail bank called Lloyds. Please assist the user answering questions with detailed responses, providing reasoning whenever prescriptive advice is given. Ensure your answers are elaborate and helpful."
            self.conversation_history.append((system_prompt, ""))
        
    
    def generate(self, query, is_user_query=True, k=3):
        vector_start_time = time.time()
        
        contexts=[]
        sources=[]
        scores=[]
        
        if is_user_query:
            # if user query, perform vector db search
            # retrieve contexts based on the query
            search_results = self.vector_store.search(query, k)
            contexts = [str(result[1]) for result in search_results]
            sources = [result[0] for result in search_results]
            scores = [result[2] for result in search_results]
            
            if sum(scores)/len(scores) < 0.65:
                for i in range(len(contexts)):
                    
                    contexts[i]='In your search you could not find relevant context within the dataset. Therefore, you are uncertain about your response. IMPORTANT: Remind the customer that you are built to answer Lloyds banking related questions only.'
                    sources[i]='Could not find relevant sources'
            
              
        vector_end_time = time.time()
        # combine both query and response from conversation history
        history_context = '\n'.join(['Q: ' + query + '\nA: ' + response for query, response in self.conversation_history[-k:]])
        
        # combine history context and current contexts
        combined_context = '\n'.join([history_context] + contexts)
        combined_context = combined_context[-20000:]
        
        lm_start_time = time.time()
        response = self.palm_wrapper.generate_response(combined_context, query)
        lm_end_time = time.time()
        # update conversation history with current interaction
        self.conversation_history.append((query, response))
        
        vector_search_time = vector_end_time - vector_start_time
        lm_inference_time = lm_end_time - lm_start_time
        
        return response, sources, scores, vector_search_time, lm_inference_time

In [10]:
rag = RAG(vai, pw)
query = "i've lost my card"
response, sources, scores, vector_search_time, lm_inference_time = rag.generate(query)
print(response)
print('sources used:', sources)
print('scores:', scores)
print('vector search time:', vector_search_time)
print('llm inference time:', lm_inference_time)

query = "why?"
response, sources, scores, vector_search_time, lm_inference_time = rag.generate(query)
print(response)
print('sources used:', sources)
print('scores:', scores)
print('vector search time:', vector_search_time)
print('llm inference time:', lm_inference_time)

query = "i was not asking about an audit request"
response, sources, score, vector_search_time, lm_inference_times = rag.generate(query)
print(response)
print('sources used:', sources)
print('scores:', scores)
print('vector search time:', vector_search_time)
print('llm inference time:', lm_inference_time)

query = "what is the weather in london"
response, sources, scores, vector_search_time, lm_inference_time = rag.generate(query)
print(response)
print('sources used:', sources)
print('scores:', scores)
print('vector search time:', vector_search_time)
print('llm inference time:', lm_inference_time)

query = "how can i set up an online banking account?"
response, sources, scores, vector_search_time, lm_inference_time = rag.generate(query)
print(response)
print('sources used:', sources)
print('scores:', scores)
print('vector search time:', vector_search_time)
print('llm inference time:', lm_inference_time)

I'm sorry to hear that you've lost your card. Please call us right away on 0800 096 4496. We'll need to cancel your card and issue you with a new one.
sources used: ['https://www.lloydsbank.com/business/help-and-support/cards/apply-for-a-business-debit-card.html', 'https://www.lloydsbank.com/business/help-and-support/security-and-fraud/report-lost-stolen-corporate-card.html', 'https://www.lloydsbank.com/business/help-and-support/online-banking/log-on-to-o4b.html?WT.ac=Lloyds-help_and_support-online_banking-button_text-log_on_to_online_for_business']
scores: [0.7142672538757324, 0.7047110795974731, 0.7031532526016235]
vector search time: 0.19745802879333496
llm inference time: 0.6073603630065918
This is to protect your account from fraud. If you have lost your card, it is important to cancel it as soon as possible so that no one else can use it. We will then issue you with a new card.
sources used: ['https://www.lloydsbank.com/business/help-and-support/security-and-fraud/report-lost-sto

In [8]:
import ipywidgets as widgets
from IPython.display import display, clear_output

class ChatUI:
    def __init__(self, rag_instance):
        self.rag_instance = rag_instance
        self.conversation = []
        self._setup_ui()
    
    def _setup_ui(self):
        self.input_box = widgets.Text(
            placeholder='Message LBG help and support...',
            description='Lloyds',
            layout={'width': '80%'}
        )
        
        self.send_button = widgets.Button(
            description='Send',
            button_style='info',
            layout={'width': '12%'}
        )
        self.output_area = widgets.Output(layout={'border': '1px solid black', 'width': '100%'})
        self.send_button.on_click(self._on_send_clicked)
        
        input_send_box = widgets.HBox([self.input_box, self.send_button])
        
        display(self.output_area, input_send_box)
        
    def _on_send_clicked(self, b):
        query = self.input_box.value
        self.conversation.append(f"You: {query}")
        response = self.rag_instance.generate(query)
        self.conversation.append(f'LBG AI: {response}')
        
        with self.output_area:
            clear_output(wait=True)
            print('\n'.join(self.conversation))
        
        # clear input box
        self.input_box.value = ''
        
rag = RAG(vai, pw)
chat_ui = ChatUI(rag)            

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…